<a href="https://colab.research.google.com/github/Prachii26/Autogluon/blob/main/colabs/AutoGluon_Tabular_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 28.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import random
from sklearn.datasets import make_regression
from datetime import datetime

In [3]:
x, y = make_regression(n_samples=100, n_features=5, n_targets=1, random_state=1)
dfx = pd.DataFrame(x, columns=['A', 'B', 'C', 'D', 'E'])
dfy = pd.DataFrame(y, columns=['label'])

# Create an integer column, a datetime column, a categorical column and a string column
dfx['B'] = (dfx['B']).astype(int)
dfx['C'] = datetime(2000, 1, 1) + pd.to_timedelta(dfx['C'].astype(int), unit='D')
dfx['D'] = pd.cut(dfx['D'] * 10, [-np.inf, -5, 0, 5, np.inf], labels=['v', 'w', 'x', 'y'])
dfx['E'] = pd.Series(list(' '.join(random.choice(["abc", "d", "ef", "ghi", "jkl"]) for i in range(4)) for j in range(100)))

dataset = TabularDataset(dfx)
print(dfx)

           A  B          C  D                E
0  -0.545774  0 2000-01-01  y     d ghi abc ef
1  -0.468674  0 2000-01-02  x  jkl ghi abc jkl
2   1.767960  0 1999-12-31  v    ghi ghi ghi d
3  -0.118771  1 2000-01-01  y    abc ghi ef ef
4   0.630196  0 1999-12-31  w   abc abc ghi ef
..       ... ..        ... ..              ...
95 -1.182318 -1 2000-01-01  v  abc ghi jkl ghi
96  0.562761  0 2000-01-01  v     ghi d abc ef
97 -0.797270  0 2000-01-01  w  ghi abc jkl jkl
98  0.502741  0 1999-12-31  y    abc d jkl ghi
99  2.056356  0 1999-12-30  w      jkl d abc d

[100 rows x 5 columns]


In [4]:
from autogluon.features.generators import AutoMLPipelineFeatureGenerator

auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
featured_data = auto_ml_pipeline_feature_generator.fit_transform(X=dfx)
print(featured_data.head())

          A  B  D    E                   C  C.year  C.month  C.day  \
0 -0.545774  0  3    3  946684800000000000    2000        1      1   
1 -0.468674  0  2  NaN  946771200000000000    2000        1      2   
2  1.767960  0  0  NaN  946598400000000000    1999       12     31   
3 -0.118771  1  3  NaN  946684800000000000    2000        1      1   
4  0.630196  0  1  NaN  946598400000000000    1999       12     31   

   C.dayofweek  E.char_count  E.symbol_ratio.   __nlp__.abc  __nlp__.ef  \
0            5             3                 3            1           1   
1            6             6                 0            1           0   
2            4             4                 2            0           0   
3            5             4                 2            1           2   
4            4             5                 1            2           1   

   __nlp__.ghi  __nlp__.jkl  __nlp__._total_  
0            1            0                3  
1            1            2       

In [5]:
df = pd.concat([dfx, dfy], axis=1)
predictor = TabularPredictor(label='label')
predictor.fit(df, hyperparameters={'GBM': {}})

No path specified. Models will be saved in: "AutogluonModels/ag-20251013_211516"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Thu Oct  2 10:42:05 UTC 2025
CPU Count:          2
Memory Avail:       11.29 GB / 12.67 GB (89.1%)
Disk Space Avail:   190.08 GB / 235.68 GB (80.7%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets=

In [6]:
# Introduce missing values
missing_data = dfx.copy()
missing_data.iloc[0, 0] = np.nan
missing_data.iloc[1, 1] = np.nan
missing_data.iloc[2, 2] = np.nan
print(missing_data.head())

          A    B          C  D                E
0       NaN  0.0 2000-01-01  y     d ghi abc ef
1 -0.468674  NaN 2000-01-02  x  jkl ghi abc jkl
2  1.767960  0.0        NaT  v    ghi ghi ghi d
3 -0.118771  1.0 2000-01-01  y    abc ghi ef ef
4  0.630196  0.0 1999-12-31  w   abc abc ghi ef


In [7]:
auto_ml_pipeline_feature_generator = AutoMLPipelineFeatureGenerator()
featured_missing_data = auto_ml_pipeline_feature_generator.fit_transform(X=missing_data)
print(featured_missing_data.head())

Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11463.31 MB
	Train Data (Original)  Memory Usage: 0.01 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
		Fitting DatetimeFeatureGenerator...
		Fitting TextSpecialFeatureGenerator...
			Fitting BinnedFeatureGenerator...
			Fitting DropDuplicatesFeatureGenerator...
		Fitting TextNgramFeatureGenerator...
			Fitting CountVectorizer for text features: ['E']
			CountVectorizer fit with vocabulary size = 4
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types o

          A    B  D    E                   C  C.year  C.month  C.day  \
0       NaN  0.0  3    3  946684800000000000    2000        1      1   
1 -0.468674  NaN  2  NaN  946771200000000000    2000        1      2   
2  1.767960  0.0  0  NaN  946688290909090944    2000        1      1   
3 -0.118771  1.0  3  NaN  946684800000000000    2000        1      1   
4  0.630196  0.0  1  NaN  946598400000000000    1999       12     31   

   C.dayofweek  E.char_count  E.symbol_ratio.   __nlp__.abc  __nlp__.ef  \
0            5             3                 3            1           1   
1            6             6                 0            1           0   
2            5             4                 2            0           0   
3            5             4                 2            1           2   
4            4             5                 1            2           1   

   __nlp__.ghi  __nlp__.jkl  __nlp__._total_  
0            1            0                3  
1            1        